In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import missingno
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import RFE

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from warnings import filterwarnings
from sklearn import ensemble
from argparse import Namespace
from sklearn.metrics import confusion_matrix as cm

filterwarnings('ignore')

from matplotlib.legend_handler import HandlerLine2D

In [ ]:
args = Namespace(target="FiyatAraligi",data_file="tablet.csv",cv = 10)

Dizinde bulunan veri setimizi tablet değişkenine atayalım ve tablet değişkenini df değişkenine kopyalayarak kullanmaya başlayalım.

In [ ]:
tablet = pd.read_csv('../input/tablet/tablet.csv')
df= tablet.copy()
df

In [ ]:
df.head()
# Veri setinin ilk 5 gözlemini görelim.

In [ ]:
df.sample(10)
# Veri setinin rastgele 10 gözlemini görüyoruz.

In [ ]:
df.tail()
# Veri setinin son 5 gözlemini görüyoruz.

In [ ]:
df.shape

In [ ]:
df.info()
# Veri setinin bilgilerini öğrenmek için info fonksiyonu kullanılıyor.
# Değişkenlerin kaçar adet olduğu, değişkenlerin bellek kullanımını ve tipini öğreniyoruz.

Bluetooth,4G,3G,Dokunmatik,CiftHat,Renk ve WiFi gibi gözlemlerin ne kadar dengeli olup olmadığını sorgulayın.

In [ ]:
df["Bluetooth"].value_counts()

In [ ]:
df["4G"].value_counts()

In [ ]:
df["3G"].value_counts()

In [ ]:
df["Dokunmatik"].value_counts()

In [ ]:
df["CiftHat"].value_counts()

In [ ]:
df["Renk"].value_counts()

In [ ]:
df["WiFi"].value_counts()

In [ ]:
df.isna().sum().sum()
#Veri setimizde totalde kaç tane değer eksik olduğunu gördük.

In [ ]:
df.isna().sum()
#Veri setindeki hangi değişkenlerden kaçar adet eksik değer olduğunu gördük.
#Ram değişkeninden 12 OnKameraMP niteliğinden 5 gözlemin eksik olduğunu gördük.

In [ ]:
df.describe().T
#Veri setindeki sayısal değişkenlerin temel istatistik değerlerini görüntüledik.
#Ortalama std gibi temel istatistik değerleri görüntülüyoruz.

In [ ]:
corr = df.corr()
corr
#Korelasyon değerlerini gözlemleyelim

In [ ]:
corr = df.corr()
sns.heatmap(corr,
           xticklabels=corr.columns.values,
           yticklabels=corr.columns.values);
#Tablomuzun neredeyse tamamına yakını siyah bu yüzden öznitelikler arasında ilişki yok denecek kadar az.
#Dikkatimizi çeken ilişkilerden birisi OnKameraMP ve ArkaKameraMP ilişkisi
#Diğeri ise CozunurlukYükseklik ve CozunurlukGenislik ilişkisi aralarındaki ilişki zayıfda olsa var.

In [ ]:
df.corr()["ArkaKameraMP"]["OnKameraMP"]
#İlişkinin en yüksek olduğu iki özniteliğin oranını görüyoruz.

In [ ]:
df.corr()["CozunurlukYükseklik"]["CozunurlukGenislik"]
#Aralarındaki ilişki 0.51 olarak görüyoruz orta güçte bir ilişki.

Veri Setinin özniteliklerini kendi içinde değerlendirelim

In [ ]:
df["FiyatAraligi"].unique
#FiyatAraligi değişkeninin 4 adet benzersiz değişkeni vardır.

In [ ]:
df["FiyatAraligi"].value_counts()
#FiyatAraligi değişkeni dengeli dağılıma sahiptir.

In [ ]:
sns.countplot(x = "FiyatAraligi", data=df);
#Eşit sayıda olduğundan dengelidir.

Diğer öznitelikleri değerlendirirken benzersiz değelerini görüp başka grafikler de kullanalım.

In [ ]:
df["ArkaKameraMP"].nunique()
#ArkaKameraMP değişkeninin 21 adet benzersiz değere sahip olduğunu öğrendik.


In [ ]:
sns.distplot(df["ArkaKameraMP"],bins=16,color="aqua");
#Değerlerin dağılımını görselleştirmesi için distplot fonk. kullandık.


In [ ]:
df["OnKameraMP"].nunique()
##OnKameraMP değişkeninin 20 adet benzersiz değere sahip olduğunu öğrendik.

In [ ]:
sns.distplot(df["OnKameraMP"],bins=16,color="aqua");

In [ ]:
df["MikroislemciHizi"].nunique()
#ArkaKameraMP değişkeninin 26 adet benzersiz değere sahip olduğunu öğrendik.

In [ ]:
sns.distplot(df["MikroislemciHizi"],bins=16,color="aqua");
#Özniteliğimiz 0.5 ile 3.0 arasında değerlere sahiptir.

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})

In [ ]:
sns.barplot(y ="CozunurlukYükseklik" ,x = "FiyatAraligi" , data = df);


Aralarında orta seviye ilişki olan OnKameraMP ve ArkaKameraMP özniteliklerinin grafiğini inceleyin.

In [ ]:
plt.figure(figsize=(18,5))
sns.lmplot(x="ArkaKameraMP",y="OnKameraMP",data=df)
plt.title('arkakameramp - onkameramp')
plt.show()
#ArkaKameraMP özniteliği artıştayken OnKameraMP özniteliğinin de arttığını doğru bir ilişki olduğunu söyleyebiliriz.

FiyatAraligi özniteliğinin değişkenlerinin dağılımını görselleştirin.İlk olarak RAM değişkeni ile ilişkisini gösterin.

In [ ]:
sns.violinplot(x="FiyatAraligi",y ="RAM", data=df);
#Fiyatı pahalı olanların RAM hızının daha fazla olduğunu görüyoruz.

In [ ]:
sns.scatterplot(x = "FiyatAraligi", y = "RAM",  data = df);

In [ ]:
sns.barplot(x="FiyatAraligi",y="CekirdekSayisi",data=df);
#Fiyat olarak aralında çok fark olmamasına rağmen FiyatAraligi pahalı olanların mikroişlemci hızları daha fazla.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "DahiliBellek", data=df);
#Bu grafikten FiyatAraligi pahalı olanların daha fazla DahiliBellek değeri olduğunu görüyoruz.

In [ ]:
sns.violinplot(x = "FiyatAraligi", y = "BataryaGucu", data=df);
#Bu grafikten FiyatAraligi pahalı olanların daha fazla CozunurlukDegeri olduğunu görüyoruz.

Data Processing Bölümü

In [ ]:
df.columns

In [ ]:
df.isnull().sum() #Eksik değerleri tekrar görelim.

In [ ]:
missingno.matrix(df,figsize=(10,10));
#Öznitelikleri görselleştirelim.

In [ ]:
missingno.heatmap(df,figsize=(6,6));
#Daha net görebilmek için ısı haritası kullandık.

In [ ]:
df = df.fillna(df.mean())
df
#Eksik değerleri ortalama değerlerle doldurmak için kullandık.

In [ ]:
df.isnull().sum()
#Tekrar kontrol edip eksik değerleri doldurabilmiş miyiz diye kontrol ettik

In [ ]:
df["Renk"].unique()
#Renk değişkenine ait benzersiz değerleri görüntüledik.

In [ ]:
df_Renk = pd.get_dummies(df["Renk"], prefix ="-",prefix_sep='Renk-', dummy_na=False, columns=None,dtype=None)
df_Renk
#get_dummies sayesinde renk e dair kategorik öznitelik çıkarımında bulunduk.
#Ordinal değil tam tersi nominaller üstünlükleri yok.

In [ ]:
df = pd.concat([df,df_Renk],axis=1)
df = df.drop("Renk",axis=1)
df
#df veri setiyle df_renk veri setini birleştirdik tek tablo haline getirdik

In [ ]:
categories=pd.Categorical(df['OnKameraMP'],ordered=True)
print(categories)
#Ordinal değerlerini görüntülüyoruz.

In [ ]:
categories=pd.Categorical(df['DahiliBellek'],ordered=True)
print(categories)

Ordinal değerleri sayısallaştıralım.

In [ ]:
df["FiyatAraligi"] = df["FiyatAraligi"].replace("Çok Ucuz",0)
df["FiyatAraligi"] = df["FiyatAraligi"].replace("Ucuz",1)
df["FiyatAraligi"] = df["FiyatAraligi"].replace("Normal",2)
df["FiyatAraligi"] = df["FiyatAraligi"].replace("Pahalı",3)

In [ ]:
df.replace("Yok",0,inplace=True)
df.replace("Var",1,inplace=True)

In [ ]:
df
#Veri setini tekrar görüntüledik

**MAKİNE ÖĞRENMESİ**

****GAUSSIAN NAIVE BAYES****

Naive Bayes sınıflandırma algoritması, adını Matematikçi Thomas Bayes’den alan bir sınıflandırma/ kategorilendirme algoritmasıdır. Naive Bayes sınıflandırması olasılık ilkelerine göre tanımlanmış bir dizi hesaplama ile, sisteme sunulan verilerin sınıfını yani kategorisini tespit etmeyi amaçlar.

In [ ]:
x=df.drop("FiyatAraligi",axis=1)
y=df["FiyatAraligi"]
#BAĞIMLI DEĞİŞKEN FİYAT ARALIĞIDIR BAĞIMSIZ DEĞİŞKENLER İSE DİĞERLERİDİR.

In [ ]:
x


In [ ]:
y

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.15,random_state=42)
# %85 eğitim %15 test olarak 4 parçaya böldük.

In [ ]:
X_train.head()

In [ ]:
Y_train.head()

In [ ]:
X_test.head()

In [ ]:
Y_test.head()

**NAIVE BAYES**

In [ ]:
nb = GaussianNB()  # modeli inşa ediyoruz.
nb_model = nb.fit(X_train, Y_train) 

In [ ]:
nb_model

Model üzerinde yazılabilecek tüm komutları görüntüleyelim.

In [ ]:
dir(nb_model)

In [ ]:
Y_pred = nb_model.predict(X_test)
# X_test verisini kullanarak Y_predi ettirelim.

In [ ]:
Y_pred 

In [ ]:
accuracy_score(Y_test, Y_pred)
#Doğruluk skorunu görüntüleyelim.

Bir karışıklık matrisi, gerçek değerlerin bilinmekte olduğu bir dizi test verisi üzerinde, bir sınıflandırma modelinin performansını tanımlamak için sıklıkla kullanılan bir tablodur. 

In [ ]:
cross_val_score(nb_model, x, y, cv = 10)

In [ ]:
cross_val_score(nb_model, x, y, cv = 10).mean()

In [ ]:
karmasiklik_matrisi = confusion_matrix(Y_test, Y_pred)
print(karmasiklik_matrisi)
#Karmaşıklık matrisini görüntüleyelim.


In [ ]:
print(classification_report(Y_test, Y_pred))
#Sınıflandırma raporu çizdirelim.

In [ ]:
PrecisionScore = precision_score(Y_test, Y_pred, average='weighted')
PrecisionScore
#Kesinlik skorunu görüntüleyelim.

In [ ]:
RecallScore = recall_score(Y_test, Y_pred, average='weighted')
RecallScore
#Yakalama (Recall) veya Hassaslık (Sensitivity) skorunu görüntüleyelim.

In [ ]:
F1Score = f1_score(Y_test, Y_pred, average = 'weighted')  
F1Score
#F1 skorunu görüntüleyelim.

**DECISION TREE (KARAR AĞACI)**

Karar Ağacı algoritması, denetimli öğrenme algoritmaları ailesine aittir. Karar Ağacı kullanmanın amacı, önceki verilerden (eğitim verileri) çıkarılan basit karar kurallarını öğrenerek hedef değişkenin sınıfını veya değerini tahmin etmek için kullanılabilecek bir eğitim modeli oluşturmaktır.

Modelimizi inşa edelim ve eğitelim.

In [ ]:
cart = DecisionTreeClassifier(random_state = 42, criterion='entropy')
cart_model = cart.fit(X_train, Y_train)
#Modelimizi inşa edelim ve eğitelim.
#Entropi rastgele değere sahip bir değişken veya sistem için belirsizlik ölçütüdür.

Modelin parametrelerini ve aldığı varsayılan değerleri görüntülemek için model ismini yazabiliriz.

In [ ]:
cart_model

Parametreleri optimize etmeden kurduğumuz bu model için tahminlerde bulunalım.

In [ ]:
Y_pred = cart_model.predict(X_test)
#Modele X_testi vererek tahmin ettirelim.

Daha önce hiç görmediği X_test verileri için bulunduğu tahminlere göre doğruluk skorunu görüntüleyelim.

In [ ]:
accuracy_score(Y_test, Y_pred)

Modelin tüm verileri öğrendiği görülmekte. Karmaşıklık matrisini görüntüleyelim.

In [ ]:
karmasiklik_matrisi = confusion_matrix(Y_test, Y_pred)
print(karmasiklik_matrisi)

Modeli farklı gözlemlerle kombinleyerek tekrardan skorunu hesaplayalım.

In [ ]:
cross_val_score(cart_model, x, y, cv = 10)

10 skorun ortalamasına bakalım.

In [ ]:
cross_val_score(cart_model, x, y, cv = 10).mean()

Hedef özniteliğin değerlerine göre sınıflandırma raporunu görüntüleyelim.

In [ ]:
print(classification_report(Y_test, Y_pred))

karar ağacı görüntüsü

In [ ]:
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
graph = Source(tree.export_graphviz(cart, out_file = None, feature_names = x.columns, filled = True))
display(SVG(graph.pipe(format = 'svg')))

In [ ]:
ranking = cart.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = x.columns

plt.figure(figsize = (10, 8))
plt.title("Karar Ağacına Göre Özniteliklerin Önem Derecesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="lime", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()

**KNN ALGORİTMASI**

k-en yakın komşu algoritması, ilk olarak 1951'de Evelyn Fix ve Joseph Hodges tarafından geliştirilen ve daha sonra Thomas Cover tarafından genişletilen parametrik olmayan bir sınıflandırma yöntemidir. Sınıflandırma ve regresyon için kullanılır. 

Veri çerçevemizi bağımlı ve bağımsız olarak bölütleyip, eğitim ve test olarak da parçalamıştık. Şimdi aynı eğitim verileri üzerinden KNN modeli kuralım ve eğitelim.

Modeli parametre optimizasyonu yaparak kuralım ve eğitelim.

In [ ]:
knn_params = {"n_neighbors": np.arange(2,50)}

 2’den 50’ye kadar olan her sayıyı komşu sayısı olarak deneyecektir.

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv = 3)
knn_cv.fit(X_train, Y_train)

In [ ]:
print("En iyi skor: " + str(knn_cv.best_score_))
print("En iyi parametreler: " + str(knn_cv.best_params_))

Doğruluk skorlarını görselleştirelim.

In [ ]:
score_list = []

for each in range(2,25):
    knn2 = KNeighborsClassifier(n_neighbors = each)
    knn2.fit(X_train,Y_train)
    score_list.append(knn2.score(X_test, Y_test))
    
plt.plot(range(2,25),score_list)
plt.xlabel("k değerleri")
plt.ylabel("doğruluk skoru")
plt.show()

En iyi komşu değerini 11 olarak bulduk bu değeri kullanarak eğitelim.


In [ ]:
knn = KNeighborsClassifier(11)
knn_tuned = knn.fit(X_train, Y_train)

In [ ]:
y_pred = knn_tuned.predict(X_test)

In [ ]:
accuracy_score(Y_test, Y_pred)

Karmasıklık matrisi.

In [ ]:
Karmasiklik_Matrisi = confusion_matrix(Y_test, Y_pred)
Karmasiklik_Matrisi

Modeli farklı gözlemlerle kombinleyerek tekrardan skorunu hesaplayalım.

In [ ]:
cross_val_score(knn_tuned, X_test, Y_test, cv = 10)

Ortalamasını alalım.

In [ ]:
cross_val_score(knn_tuned, X_test, Y_test, cv = 10).mean()

Hedef özniteliğin değerlerine göre sınıflandırma raporunu görüntüleyelim.

In [ ]:
print(classification_report(Y_test, Y_pred))